In [1]:
import urllib
import os
from requests import get
from matplotlib import pyplot
from bs4 import BeautifulSoup
%matplotlib inline

from osgeo import gdal
import rasterio
import rasterio.plot

import numpy as np
from PIL import Image
import imageio

In [2]:
#전역변수 지정 (연구실용)
url = "https://www.cs.toronto.edu/~vmnih/data/"

train_input_url = "mass_roads/train/sat/index.html"
train_target_url = "mass_roads/train/map/index.html"

train_directory = "C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2. Road extraction/datasets/train/input"
target_directory = "C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2. Road extraction/datasets/train/target"

dir_train = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2. Road extraction/datasets/train/input_filtered'
dir_target = 'C:/Users/spinsi/Desktop/code/jupyternotebook_python37/2. Road extraction/datasets/train/target_filtered'

In [45]:
#전역변수 지정 (노트북용)
url = "https://www.cs.toronto.edu/~vmnih/data/"

train_input_url = "mass_roads/train/sat/index.html"
train_target_url = "mass_roads/train/map/index.html"

train_directory = "C:/Users/Jiyong/Desktop/coding/Python/DeepLearning/datasets/train/input"
target_directory = "C:/Users/Jiyong/Desktop/coding/Python/DeepLearning/datasets/train/target"

dir_train = 'C:/Users/Jiyong/Desktop/coding/Python/DeepLearning/datasets/train/input_filtered'
dir_target = 'C:/Users/Jiyong/Desktop/coding/Python/DeepLearning/datasets/train/target_filtered'

In [3]:
# 데이터 다운받기
class downloader:
    def __init__(self, url, url2, directory):
        self.url = url
        self.url2 = url2
        self.directory = directory
        
    def parse(self):
        req = urllib.request.Request(self.url+self.url2)
        sourcecode=urllib.request.urlopen(self.url+self.url2).read()
        soup = BeautifulSoup(sourcecode, "html.parser")
        cell_line = []
        links = soup.find_all('a')
        for i in links:
            href = i.attrs['href']
            cell_line.append(href)
            for j in cell_line:
                if j.find("index") != -1:
                    cell_line.remove(j)
        os.chdir(self.directory)
        for j in cell_line:
            urllib.request.urlretrieve(j, j[59:])

In [4]:
dl = downloader(url,train_input_url,train_directory)
dl.parse()


KeyboardInterrupt: 

In [ ]:
# 이미지 전처리
# adjust band 모듈은 0 ~ 1 사이로 normalization 하는 모듈
# 일단 이미지 파일과 라벨 파일이 다른 것에 대해서 잘라냄
for a in os.listdir(target_directory):
    
    os.chdir(train_directory)
    train = rasterio.open(a+'f')
    
    os.chdir(target_directory)
    target = rasterio.open(a)
    
    trainimg = np.array([(train.read(i)) for i in (1,2,3)])
    targetimg = np.array([(target.read(1))])
    
    for i in range(1500):
        for j in range(1500):
            if trainimg[0, i, j] == 255 and trainimg[1, i, j] == 255 and trainimg[2, i, j] == 255:
                targetimg[0, i, j] = 0
    
    profile = target.profile
    dataset = rasterio.open(a, 'w', **profile)
    dataset.write(targetimg.astype(rasterio.uint8))
    
# 일단은 파일 중에서 흰색 화면의 비율이 높은 파일 제거
# 그다음은 augmentation을 통해서 데이터 갯수 늘이는 것이 목적
# 파일이 워낙 많아서 처리하기가 워낙 쉬운게 아니네 좀 더 빠른 방법은 없을까?

In [12]:
# 이미지 중 흰색 비율이 높은 것들에 대해서 삭제
for a in os.listdir(target_directory):
    
    os.chdir(train_directory)
    train = rasterio.open(a+'f')
    
    os.chdir(target_directory)
    target = rasterio.open(a)

    trainimg = np.array([(train.read(i)) for i in (1,2,3)])
    trainimg = trainimg.transpose(1,2,0)
    targetimg = np.array([(target.read(1))])
    targetimg = targetimg.transpose(1,2,0)
    
    count = np.count_nonzero(np.all(trainimg == [255,255,255], axis=-1))
    if count <= 337500:
        os.chdir(dir_train)
        imageio.imwrite(a[:-4]+'.png',trainimg)
        os.chdir(dir_target)
        imageio.imwrite(a[:-4]+'.png',targetimg)
        
    print(a)

10078660_15.tif
10078675_15.tif
10078690_15.tif
10078705_15.tif
10078720_15.tif
10078735_15.tif
10078750_15.tif
10228660_15.tif
10228675_15.tif
10228705_15.tif
10228720_15.tif
10228735_15.tif
10228750_15.tif
10228765_15.tif
10228780_15.tif
10228795_15.tif
10378660_15.tif
10378675_15.tif
10378690_15.tif
10378705_15.tif
10378720_15.tif
10378735_15.tif
10378750_15.tif
10378765_15.tif
10378795_15.tif
10528660_15.tif
10528675_15.tif
10528690_15.tif
10528705_15.tif
10528720_15.tif
10528735_15.tif
10528750_15.tif
10528765_15.tif
10528780_15.tif
10528795_15.tif
10678660_15.tif
10678675_15.tif
10678690_15.tif
10678705_15.tif
10678720_15.tif
10678735_15.tif
10678750_15.tif
10678765_15.tif
10678780_15.tif
10678795_15.tif
10678810_15.tif
10678825_15.tif
10678870_15.tif
10678885_15.tif
10828645_15.tif
10828660_15.tif
10828675_15.tif
10828690_15.tif
10828705_15.tif
10828735_15.tif
10828750_15.tif
10828765_15.tif
10828780_15.tif
10828795_15.tif
10828810_15.tif
10828825_15.tif
10828855_15.tif
10828870

21328870_15.tif
21328885_15.tif
21328900_15.tif
21328915_15.tif
21328930_15.tif
21328945_15.tif
21328960_15.tif
21328975_15.tif
21328990_15.tif
21329005_15.tif
21329020_15.tif
21329035_15.tif
21329050_15.tif
21478870_15.tif
21478885_15.tif
21478900_15.tif
21478915_15.tif
21478930_15.tif
21478945_15.tif
21478960_15.tif
21478975_15.tif
21478990_15.tif
21479005_15.tif
21479020_15.tif
21479050_15.tif
21479065_15.tif
21628870_15.tif
21628885_15.tif
21628900_15.tif
21628915_15.tif
21628930_15.tif
21628945_15.tif
21628960_15.tif
21628975_15.tif
21628990_15.tif
21629005_15.tif
21629020_15.tif
21629035_15.tif
21629050_15.tif
21629065_15.tif
21778900_15.tif
21778915_15.tif
21778930_15.tif
21778945_15.tif
21778960_15.tif
21778975_15.tif
21778990_15.tif
21779020_15.tif
21779035_15.tif
21779050_15.tif
21779065_15.tif
21779080_15.tif
21928900_15.tif
21928915_15.tif
21928930_15.tif
21928945_15.tif
21928960_15.tif
21928975_15.tif
21928990_15.tif
21929005_15.tif
21929035_15.tif
21929050_15.tif
21929065

RasterioIOError: TIFFReadDirectory:Failed to read directory at offset 8

In [35]:

b = os.listdir(target_directory)
os.chdir(train_directory)
train = rasterio.open(b[0]+'f')

os.chdir(target_directory)
target = rasterio.open(b[0])

trainimg = np.array([(train.read(i)) for i in (1,2,3)])
trainimg = trainimg.transpose(1,2,0)
targetimg = np.array([(target.read(1))])
targetimg = targetimg.transpose(1,2,0)
targetimg = targetimg.reshape((1500,1500))


In [36]:
print(np.shape(targetimg))

(1500, 1500)


In [39]:
os.chdir(dir_train)
Image.fromarray(trainimg).save("1.png")

os.chdir(dir_target)
Image.fromarray(targetimg).save("2.png")

In [41]:
os.chdir(dir_train)
imgArray=np.array(Image.open("1.png"))
print(np.unique(imgArray))

os.chdir(dir_target)
imgArray2=np.array(Image.open("2.png"))
print(np.unique(imgArray2))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24